In [1]:
from pathlib import Path
from __future__ import annotations
import yaml
import pandas as pd
import requests
from io import StringIO
import numpy as np
from scipy.io import loadmat
import plotly.express as px
import plotly.graph_objects as go
import os
import h5py

from mda_core import constants as C, utils
from mda_core.attitude import laws
from mda_core.propagation import  propagator
from mda_core.time import Time

from org.orekit.frames import FramesFactory
from org.orekit.utils import TimeStampedPVCoordinates
from org.orekit.propagation import SpacecraftState
from org.orekit.orbits import CartesianOrbit
from mda_core import frames
from org.orekit.utils import IERSConventions, PVCoordinates
from mda_core.attitude.attitude_providers import ManueverPointingL1,EclipsePointingL1
from org.hipparchus.geometry.euclidean.threed import Rotation,Vector3D

# Query files from HTTP Server

In [2]:
month = '09'
day = '19'
hour = '17'
min = '18'
sec = '08'
run_no = '3'

sub_folder_path = f'../results/logs_2024_{month}_{day}_{hour}_{min}_{sec}/'

# Create the folder if it doesn't exist
os.makedirs(sub_folder_path, exist_ok=True)

# All Basilisk files required for Analysis later
url1 = f'http://localhost:8000/RefProfPathway{run_no}/logs_2024_{month}_{day}-{hour}_{min}_{sec}/csv/FswModels.aocsLayerModule.aocsDataMsg.q_BodyInertial_ref.csv'
url2 = f'http://localhost:8000/RefProfPathway{run_no}/logs_2024_{month}_{day}-{hour}_{min}_{sec}/csv/FswModels.aocsLayerModule.aocsDataMsg.v_satPos_Inertial_est.csv'
url3 = f'http://localhost:8000/RefProfPathway{run_no}/logs_2024_{month}_{day}-{hour}_{min}_{sec}/csv/FswModels.aocsLayerModule.aocsDataMsg.v_satVel_Inertial_est.csv'
url4 = f'http://localhost:8000/RefProfPathway{run_no}/logs_2024_{month}_{day}-{hour}_{min}_{sec}/csv/FswModels.aocsLayerModule.aocsDataMsg.v_satPosJ2_calc.csv'
url5 = f'http://localhost:8000/RefProfPathway{run_no}/logs_2024_{month}_{day}-{hour}_{min}_{sec}/csv/FswModels.aocsLayerModule.aocsDataMsg.v_satVelJ2_calc.csv'
url6 = f'http://localhost:8000/RefProfPathway{run_no}/logs_2024_{month}_{day}-{hour}_{min}_{sec}/csv/FswModels.aocsLayerModule.aocsDataMsg.s_JulianDay_ref.csv'
url7 = f'http://localhost:8000/RefProfPathway{run_no}/logs_2024_{month}_{day}-{hour}_{min}_{sec}/csv/FswModels.aocsLayerModule.aocsDataMsg.s_numberOperation.csv'

In [3]:
# Fetch the CSV files
response1 = requests.get(url1, verify=False)  # verify=False to bypass SSL verification if needed
response2 = requests.get(url2, verify=False)
response3 = requests.get(url3, verify=False)
response4 = requests.get(url4, verify=False)
response5 = requests.get(url5, verify=False)
response6 = requests.get(url6, verify=False)
response7 = requests.get(url7, verify=False)

# Check if the requests were successful
if response1.status_code == 200 and response2.status_code == 200 and response3.status_code == 200 and response4.status_code == 200 and response5.status_code == 200 and response6.status_code == 200:
    # Convert the content to DataFrames
    df_basilisk = pd.read_csv(StringIO(response1.text))
    df3 = pd.read_csv(StringIO(response2.text))
    df4 = pd.read_csv(StringIO(response3.text))
    df2 = pd.read_csv(StringIO(response4.text))
    df6 = pd.read_csv(StringIO(response5.text))
    df7 = pd.read_csv(StringIO(response6.text))
    df10 = pd.read_csv(StringIO(response7.text))
else:
    print(f"Failed to fetch CSV files!")

In [4]:
df_basilisk

,# Time (s),q_BodyInertial_ref[0],q_BodyInertial_ref[1],q_BodyInertial_ref[2],q_BodyInertial_ref[3]
0,0.0,1.000000,0.000000,0.000000,0.000000
1,0.1,1.000000,0.000000,0.000000,0.000000
2,0.2,1.000000,0.000000,0.000000,0.000000
3,0.3,1.000000,0.000000,0.000000,0.000000
4,0.4,1.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...
3596,359.6,0.180732,0.393434,0.299102,0.850344
3597,359.7,0.180676,0.393477,0.299090,0.850340
3598,359.8,0.180619,0.393519,0.299078,0.850336
3599,359.9,0.180563,0.393562,0.299067,0.850332


In [5]:
df10

,# Time (s),s_numberOperation[0]
0,0.0,0.0
1,0.1,6.0
2,0.2,6.0
3,0.3,6.0
4,0.4,6.0
...,...,...
3596,359.6,3.0
3597,359.7,3.0
3598,359.8,3.0
3599,359.9,3.0


In [6]:
df_basilisk['Operation'] = df10['s_numberOperation[0]']
df_basilisk

,# Time (s),q_BodyInertial_ref[0],q_BodyInertial_ref[1],q_BodyInertial_ref[2],q_BodyInertial_ref[3],Operation
0,0.0,1.000000,0.000000,0.000000,0.000000,0.0
1,0.1,1.000000,0.000000,0.000000,0.000000,6.0
2,0.2,1.000000,0.000000,0.000000,0.000000,6.0
3,0.3,1.000000,0.000000,0.000000,0.000000,6.0
4,0.4,1.000000,0.000000,0.000000,0.000000,6.0
...,...,...,...,...,...,...
3596,359.6,0.180732,0.393434,0.299102,0.850344,3.0
3597,359.7,0.180676,0.393477,0.299090,0.850340,3.0
3598,359.8,0.180619,0.393519,0.299078,0.850336,3.0
3599,359.9,0.180563,0.393562,0.299067,0.850332,3.0


In [7]:
df6['# Time (s)'] = df6['# Time (s)'].astype('float64')
df6

,# Time (s),v_satVelJ2_calc[0],v_satVelJ2_calc[1],v_satVelJ2_calc[2]
0,0.0,0.000000,0.000000,0.000000
1,0.1,0.000000,0.000000,0.000000
2,0.2,0.000000,0.000000,0.000000
3,0.3,0.000000,0.000000,0.000000
4,0.4,0.000000,0.000000,0.000000
...,...,...,...,...
3596,359.6,2484.416134,1750.615371,6928.349564
3597,359.7,2485.155291,1750.798582,6928.038636
3598,359.8,2485.894121,1750.981699,6927.727751
3599,359.9,2486.633219,1751.164869,6927.416660


In [8]:
df2

,# Time (s),v_satPosJ2_calc[0],v_satPosJ2_calc[1],v_satPosJ2_calc[2]
0,0.0,0.000000e+00,0.000000e+00,0.000000e+00
1,0.1,0.000000e+00,0.000000e+00,0.000000e+00
2,0.2,0.000000e+00,0.000000e+00,0.000000e+00
3,0.3,0.000000e+00,0.000000e+00,0.000000e+00
4,0.4,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...
3596,359.6,-6.261060e+06,-1.551954e+06,2.626177e+06
3597,359.7,-6.260811e+06,-1.551779e+06,2.626870e+06
3598,359.8,-6.260563e+06,-1.551604e+06,2.627563e+06
3599,359.9,-6.260314e+06,-1.551429e+06,2.628255e+06


# Plot Reference Profile Q Error against Orekit

## Generate Orekit q's

In [9]:
data = {
  "epoch": "2022-05-01T16:47:50",
  "time_step": 0.1,
  "duration": 360,
  "initial_position": {
    "x": -6660190.6221287325,
    "y": -2048579.8185718404,
    "z": 2.3283064365386963e-10
  },
  "initial_velocity": {
    "vx": -292.1105417444446,
    "vy": 976.9787086991639,
    "vz": 7494.238898865543
  },
  "operation_changes": [
    # { 
    #   "time": "2022-05-01T16:49:00",
    #   "operation_number": 7,
    #   "sunlit_status": 0,
    #   "pathway":2,
    #   "u1":[0.7727280528,0.2142570623,0.4108934332],
    #   "u2":[0.1594123506,0.671248554,0.9256773739],
    #   "v1":[0.8759942102,0.8174424877,0.3384834078],
    #   "v2":[0.9898995846, 0.9250801576, 0.4289896406]
    # },
    {
      "time": "2022-05-01T16:49:00",
      "operation_number": 2,
      "pathway":3,
      "sunlit_status":0
    },
    {
      "time": "2022-05-01T16:51:00",
      "operation_number": 3,
      "pathway":3,
      "latitude": 78.2315,
      "longitude":15.4111
    }
    # {
    #    "time": "2022-05-01T16:51:00",
    #   "operation_number": 7,
    #   "pathway":3,
    #   "maneuver_direction": 'positive'
    # }
    # {
    #   "time": "2022-05-01T16:50:00",
    #   "operation_number": 2,
    #   "pathway": 3,
    #   "roll_angle": 20.0
    # }
    
  ]
}

In [10]:
eme_frame = FramesFactory.getEME2000()

ITRF = FramesFactory.getITRF(IERSConventions.IERS_2010, False)

In [11]:
config_file = Path("../../mda-core/tests/propagation/config_firefly.yml")
config = yaml.load(config_file.read_text(), yaml.FullLoader)

In [12]:
time  = data['epoch']
time_step = 0.1
p_eci = utils.list_to_vector3d([data['initial_position']['x'],data['initial_position']['y'],data['initial_position']['z']])
v_eci = utils.list_to_vector3d([data['initial_velocity']['vx'],data['initial_velocity']['vy'],data['initial_velocity']['vz']])

In [13]:
epoch= Time(time)
duration = data['duration']
time_step = data['time_step']
PV_eci  =PVCoordinates(p_eci,v_eci)
orbit = CartesianOrbit(TimeStampedPVCoordinates(epoch.get_absolutedate(),PV_eci),frames.get_earth_J2000(),C.EARTH_MU)
true_prop = propagator.get_numerical_propagator(SpacecraftState(orbit), config, orbit_type="cartesian")

2024-09-19 17:21:04.987 | DEBUG    | mda_core.propagation.propagator:_get_integrator:85 - Added Dormand Prince 853 integrator for propagation


2024-09-19 17:21:05.124 | DEBUG    | mda_core.propagation.propagator:_get_numerical_forces_from_config:148 - Added ITRF frame as earth body fixed frame
2024-09-19 17:21:05.126 | DEBUG    | mda_core.propagation.propagator:_get_numerical_forces_from_config:156 - Added gravity harmonics (model : eigen-6s, d x o : 33x33), for the body earth
2024-09-19 17:21:05.128 | DEBUG    | mda_core.propagation.propagator:_get_numerical_forces_from_config:181 - Added third body attraction force contribution from SUN
2024-09-19 17:21:05.128 | DEBUG    | mda_core.propagation.propagator:_get_numerical_forces_from_config:181 - Added third body attraction force contribution from MOON
2024-09-19 17:21:05.130 | DEBUG    | mda_core.propagation.propagator:_get_numerical_forces_from_config:181 - Added third body attraction force contribution from MARS
2024-09-19 17:21:05.132 | DEBUG    | mda_core.propagation.propagator:_get_numerical_forces_from_config:181 - Added third body attraction force contribution from JUP

In [14]:
def get_atttitude_profile(mode_description):
    operation_number = mode_description['operation_number']
    if operation_number ==1:
        if mode_description['sunlit_status']==1:
            return laws.get_sun_pointing_with_nadir_phasing(sun_pointing_axis=[0,-1,0],nadir_phasing_axis=[0,0,1])
        else:
            return EclipsePointingL1(sun_pointing_axis=[0,-1,0],nadir_phasing_axis=[0,0,1])
    if operation_number ==2:
        if mode_description['sunlit_status']==1:
            return laws.get_sun_pointing_with_nadir_phasing(sun_pointing_axis=[0,-1,0],nadir_phasing_axis=[0,0,1])
        else:
            return EclipsePointingL1(sun_pointing_axis=[0,-1,0],nadir_phasing_axis=[0,0,1])
    if operation_number == 3:
        return laws.get_gs_pointing(latitude=mode_description['latitude'],longitude=mode_description['latitude'],altitude=484.0,yaw_steering=True)
    if operation_number == 4:
        return laws.get_off_nadir_pointing(roll_angle=mode_description['roll_angle'])
    if operation_number == 6:
        if mode_description['sunlit_status']==1:
            return laws.get_sun_pointing_with_nadir_phasing(sun_pointing_axis=[0,-1,0],nadir_phasing_axis=[0,0,1])
        else:
            return EclipsePointingL1(sun_pointing_axis=[0,-1,0],nadir_phasing_axis=[0,0,1])
    if operation_number == 7:
        return ManueverPointingL1(star_tracker_dir=[0, 0.766, -0.642],manuever_dir=mode_description['maneuver_direction'])

In [15]:
def append_TQ_to_dict(dict,operation, Rotation, t, truestate):
    dict["time"].append(t.get_datetime())
    dict["operation"].append(operation)
    dict["q0"].append(Rotation.getQ0())
    dict["q1"].append(Rotation.getQ1())
    dict["q2"].append(Rotation.getQ2())
    dict["q3"].append(Rotation.getQ3())
    dict["px"].append(truestate.getPVCoordinates().getPosition().getX())
    dict["py"].append(truestate.getPVCoordinates().getPosition().getY())
    dict["pz"].append(truestate.getPVCoordinates().getPosition().getZ())
    dict["vx"].append(truestate.getPVCoordinates().getVelocity().getX())
    dict["vy"].append(truestate.getPVCoordinates().getVelocity().getY())
    dict["vz"].append(truestate.getPVCoordinates().getVelocity().getZ())

quaternion_data = {
    "time":[],
    "operation":[],
    "q0": [],
    "q1": [],
    "q2": [],
    "q3": [],
    "px": [],
    "py": [],
    "pz": [],
    "vx": [],
    "vy": [],
    "vz": [],
}

In [16]:
num_operation = len(data['operation_changes'])
if num_operation == 1:
    epoch_start = Time(data['operation_changes'][0]['time'])
    epoch_end = epoch+duration
    operation_number = data['operation_changes'][0]['operation_number']
    time_iterator= Time.iterator(epoch_start, epoch_end, time_step)
    for t in time_iterator:
        true_state= true_prop.propagate(t.get_absolutedate())
        rot = get_atttitude_profile(data['operation_changes'][0]).getAttitude(true_prop,t.get_absolutedate(),frames.get_earth_J2000()).getRotation()
        if data['operation_changes'][0]['pathway'] ==2:
            R = Rotation(utils.list_to_vector3d(data['operation_changes'][0]['u1']),utils.list_to_vector3d(data['operation_changes'][0]['u2']),utils.list_to_vector3d(data['operation_changes'][0]['v1']),utils.list_to_vector3d(data['operation_changes'][0]['v2']))
            rot = R.applyTo(rot)
        append_TQ_to_dict(quaternion_data,operation_number,rot,t,true_state)
else:
    for i in range(len(data['operation_changes'])-1):
        epoch_start = Time(data['operation_changes'][i]['time'])
        epoch_end = Time(data['operation_changes'][i+1]['time'])-time_step
        operation_number = data['operation_changes'][i]['operation_number']
        time_iterator= Time.iterator(epoch_start, epoch_end, time_step)
        for t in time_iterator:
            true_state= true_prop.propagate(t.get_absolutedate())
            rot = get_atttitude_profile(data['operation_changes'][i]).getAttitude(true_prop,t.get_absolutedate(),frames.get_earth_J2000()).getRotation()
            if data['operation_changes'][0]['pathway'] ==2:
                R = Rotation(utils.list_to_vector3d(data['operation_changes'][i]['u1']),utils.list_to_vector3d(data['operation_changes'][i]['u2']),utils.list_to_vector3d(data['operation_changes'][i]['v1']),utils.list_to_vector3d(data['operation_changes'][i]['v2']))
                rot = R.applyTo(rot)
            append_TQ_to_dict(quaternion_data,operation_number,rot,t,true_state)
    epoch_start = Time(data['operation_changes'][num_operation-1]['time'])
    epoch_end = epoch+duration
    operation_number = data['operation_changes'][num_operation-1]['operation_number']
    print(operation_number)
    time_iterator= Time.iterator(epoch_start, epoch_end, time_step)
    for t in time_iterator:
        true_state= true_prop.propagate(t.get_absolutedate())
        rot = get_atttitude_profile(data['operation_changes'][num_operation-1]).getAttitude(true_prop,t.get_absolutedate(),frames.get_earth_J2000()).getRotation()
        if data['operation_changes'][0]['pathway'] ==2:
            R = Rotation(utils.list_to_vector3d(data['operation_changes'][num_operation-1]['u1']),utils.list_to_vector3d(data['operation_changes'][num_operation-1]['u2']),utils.list_to_vector3d(data['operation_changes'][num_operation-1]['v1']),utils.list_to_vector3d(data['operation_changes'][num_operation-1]['v2']))
            rot = R.applyTo(rot)
        append_TQ_to_dict(quaternion_data,operation_number,rot,t,true_state)
df_orekit = pd.DataFrame(quaternion_data)

3


In [17]:
df_orekit

,time,operation,q0,q1,q2,q3,px,py,pz,vx,vy,vz
0,2022-05-01 16:49:00.000,2,-0.332746,0.715020,-0.162281,0.593036,-6.661377e+06,-1.974339e+06,5.240901e+05,258.249282,1143.166043,7472.528932
1,2022-05-01 16:49:00.100,2,-0.332759,0.715044,-0.162339,0.592983,-6.661351e+06,-1.974225e+06,5.248373e+05,259.035280,1143.398978,7472.466885
2,2022-05-01 16:49:00.200,2,-0.332772,0.715069,-0.162396,0.592931,-6.661325e+06,-1.974111e+06,5.255846e+05,259.821276,1143.631899,7472.404749
3,2022-05-01 16:49:00.300,2,-0.332785,0.715093,-0.162454,0.592878,-6.661299e+06,-1.973996e+06,5.263318e+05,260.607268,1143.864807,7472.342524
4,2022-05-01 16:49:00.400,2,-0.332798,0.715118,-0.162511,0.592826,-6.661273e+06,-1.973882e+06,5.270790e+05,261.393258,1144.097701,7472.280211
...,...,...,...,...,...,...,...,...,...,...,...,...
2896,2022-05-01 16:53:49.600,3,0.168757,0.417183,0.199856,0.870366,-6.260864e+06,-1.551814e+06,2.626725e+06,2484.998344,1750.759235,6928.103551
2897,2022-05-01 16:53:49.700,3,0.168710,0.417223,0.199837,0.870360,-6.260615e+06,-1.551639e+06,2.627417e+06,2485.737322,1750.942390,6927.792638
2898,2022-05-01 16:53:49.800,3,0.168662,0.417264,0.199819,0.870354,-6.260367e+06,-1.551464e+06,2.628110e+06,2486.476271,1751.125524,6927.481643
2899,2022-05-01 16:53:49.900,3,0.168615,0.417304,0.199800,0.870348,-6.260118e+06,-1.551289e+06,2.628803e+06,2487.215190,1751.308637,6927.170566


## Load the Orekit q file

In [18]:
# Load the CSV file into a DataFrame
# filepathh = f'../data/qref.csv'
# df_orekit = pd.read_csv(filepathh)
error = pd.DataFrame()
start_epoch = Time("2022-05-01T16:47:50")

## Process both CSV

In [19]:
epoch = []
start_specific_timestamp = pd.Timestamp('2022-05-01 16:50:00.000')
end_specific_timestamp = pd.Timestamp('2022-05-01 17:07:50.000')
time = start_epoch
for i in range (len(df_basilisk)):
    time = start_epoch + df_basilisk['# Time (s)'][i]
    epoch.append(pd.Timestamp(time.get_datetime()))
df_basilisk['# Time (s)'] = epoch
# df_basilisk = df_basilisk[(df_basilisk['# Time (s)'] >= start_specific_timestamp) & (df_basilisk['# Time (s)'] <= end_specific_timestamp)]
df_basilisk = df_basilisk.reset_index(drop=True)
df_basilisk['quaternion'] = df_basilisk.apply(lambda row: (row['q_BodyInertial_ref[0]'], row['q_BodyInertial_ref[1]'], row['q_BodyInertial_ref[2]'], row['q_BodyInertial_ref[3]']), axis=1)
df_basilisk = df_basilisk.drop(columns=['q_BodyInertial_ref[0]','q_BodyInertial_ref[1]','q_BodyInertial_ref[2]','q_BodyInertial_ref[3]'])
df_basilisk

,# Time (s),Operation,quaternion
0,2022-05-01 16:47:50.000,0.0,"(1.0, 0.0, 0.0, 0.0)"
1,2022-05-01 16:47:50.100,6.0,"(1.0, 0.0, 0.0, 0.0)"
2,2022-05-01 16:47:50.200,6.0,"(1.0, 0.0, 0.0, 0.0)"
3,2022-05-01 16:47:50.300,6.0,"(1.0, 0.0, 0.0, 0.0)"
4,2022-05-01 16:47:50.400,6.0,"(1.0, 0.0, 0.0, 0.0)"
...,...,...,...
3596,2022-05-01 16:53:49.600,3.0,"(0.18073168897396064, 0.3934337363151429, 0.29..."
3597,2022-05-01 16:53:49.700,3.0,"(0.18067550425533344, 0.3934765952510808, 0.29..."
3598,2022-05-01 16:53:49.800,3.0,"(0.18061932863924168, 0.3935194421147498, 0.29..."
3599,2022-05-01 16:53:49.900,3.0,"(0.1805631169099877, 0.39356231139137776, 0.29..."


In [20]:
# Function to check and adjust quaternion sign
def adjust_quaternion(row):
    if row['q0'] < 0:
        return [-row['q0'], -row['q1'], -row['q2'], -row['q3']]
    else:
        return [row['q0'], row['q1'], row['q2'], row['q3']]

In [21]:
epoch1 = []
q0 = []
q = []
for i in range(len(df_orekit)):
    t = df_orekit['time'][i]
    epoch1.append(pd.Timestamp(t))
df_orekit['time'] = epoch1
# df_orekit = df_orekit[(df_orekit['time'] >= start_specific_timestamp) & (df_orekit['time'] <= end_specific_timestamp)]
df_orekit = df_orekit.reset_index(drop=True)
# dff = pd.DataFrame()
# dff = df_orekit[['q0', 'q1', 'q2', 'q3']]
# df_orekit['quaternion'] = df_orekit.apply(lambda row: (row['q0'], row['q1'], row['q2'], row['q3']), axis=1)
df_orekit['Quaternion'] = df_orekit.apply(adjust_quaternion, axis = 1)
df_orekit['Quaternion_conjugate'] = df_orekit['Quaternion'].apply(lambda q: np.array([q[0], -q[1], -q[2], -q[3]]))
# df_orekit = df_orekit.drop(df_orekit.index[-1])
df_orekit = df_orekit.drop(columns=['q0','q1','q2','q3'])
df_orekit

,time,operation,px,py,pz,vx,vy,vz,Quaternion,Quaternion_conjugate
0,2022-05-01 16:49:00.000,2,-6.661377e+06,-1.974339e+06,5.240901e+05,258.249282,1143.166043,7472.528932,"[0.33274616344217567, -0.7150199292146033, 0.1...","[0.33274616344217567, 0.7150199292146033, -0.1..."
1,2022-05-01 16:49:00.100,2,-6.661351e+06,-1.974225e+06,5.248373e+05,259.035280,1143.398978,7472.466885,"[0.33275921572583744, -0.7150443320666561, 0.1...","[0.33275921572583744, 0.7150443320666561, -0.1..."
2,2022-05-01 16:49:00.200,2,-6.661325e+06,-1.974111e+06,5.255846e+05,259.821276,1143.631899,7472.404749,"[0.3327722490821401, -0.7150687318893885, 0.16...","[0.3327722490821401, 0.7150687318893885, -0.16..."
3,2022-05-01 16:49:00.300,2,-6.661299e+06,-1.973996e+06,5.263318e+05,260.607268,1143.864807,7472.342524,"[0.33278526351913523, -0.7150931286848025, 0.1...","[0.33278526351913523, 0.7150931286848025, -0.1..."
4,2022-05-01 16:49:00.400,2,-6.661273e+06,-1.973882e+06,5.270790e+05,261.393258,1144.097701,7472.280211,"[0.3327982590448696, -0.7151175224548967, 0.16...","[0.3327982590448696, 0.7151175224548967, -0.16..."
...,...,...,...,...,...,...,...,...,...,...
2896,2022-05-01 16:53:49.600,3,-6.260864e+06,-1.551814e+06,2.626725e+06,2484.998344,1750.759235,6928.103551,"[0.16875716886500877, 0.4171825123486509, 0.19...","[0.16875716886500877, -0.4171825123486509, -0...."
2897,2022-05-01 16:53:49.700,3,-6.260615e+06,-1.551639e+06,2.627417e+06,2485.737322,1750.942390,6927.792638,"[0.16870968038079007, 0.4172230074156076, 0.19...","[0.16870968038079007, -0.4172230074156076, -0...."
2898,2022-05-01 16:53:49.800,3,-6.260367e+06,-1.551464e+06,2.628110e+06,2486.476271,1751.125524,6927.481643,"[0.1686621860205569, 0.41726350445575905, 0.19...","[0.1686621860205569, -0.41726350445575905, -0...."
2899,2022-05-01 16:53:49.900,3,-6.260118e+06,-1.551289e+06,2.628803e+06,2487.215190,1751.308637,6927.170566,"[0.16861468578428412, 0.41730400346946056, 0.1...","[0.16861468578428412, -0.41730400346946056, -0..."


## Save the file to the results folder

In [22]:
file_path = os.path.join(sub_folder_path, 'Basilisk_q.csv')
file_path_orekit = os.path.join(sub_folder_path,'Orekit_q.csv')

df_basilisk.to_csv(file_path, index=False)
df_orekit.to_csv(file_path_orekit, index=False)

## Plot the error!

In [23]:
def error_angle_generator(q, r):
    # Extract the values from Q0
    w0 = q[0]
    x0 = q[1]
    y0 = q[2]
    z0 = q[3]
     
    # Extract the values from Q1
    w1 = r[0]
    x1 = r[1]
    y1 = r[2]
    z1 = r[3]
     
    # Computer the product of the two quaternions, term by term
    q12_w = w0 * w1 - x0 * x1 - y0 * y1 - z0 * z1
    q12_x = w0 * x1 + x0 * w1 + y0 * z1 - z0 * y1
    q12_y = w0 * y1 - x0 * z1 + y0 * w1 + z0 * x1
    q12_z = w0 * z1 + x0 * y1 - y0 * x1 + z0 * w1

    q12 = np.array([q12_w, q12_x, q12_y, q12_z])
    angle_error = 2 * (np.arctan2(np.linalg.norm(q12[1:3]),q12[0]) * (180/np.pi))
    if angle_error <= 180:
        angle_error = angle_error
    elif angle_error > 180:
        angle_error = 360 - angle_error

        
    return angle_error

In [24]:
merged_df = pd.DataFrame()
merged_df = pd.merge_asof(df_orekit,df_basilisk,left_on='time',right_on='# Time (s)')

merged_df.columns

Index(['time', 'operation', 'px', 'py', 'pz', 'vx', 'vy', 'vz', 'Quaternion',
       'Quaternion_conjugate', '# Time (s)', 'Operation', 'quaternion'],
      dtype='object')

In [25]:
error1 = []
for i in range(len(merged_df)):
    q1 = merged_df['quaternion'][i]
    q2 = merged_df['Quaternion_conjugate'][i]
    angle_error = error_angle_generator(q2,q1)
    error1.append(angle_error)
error['Time'] = merged_df['time']
error['angle error'] = error1
error['operation'] = merged_df['Operation']
error 

,Time,angle error,operation
0,2022-05-01 16:49:00.000,131.180571,1.0
1,2022-05-01 16:49:00.100,131.181029,2.0
2,2022-05-01 16:49:00.200,0.002572,2.0
3,2022-05-01 16:49:00.300,0.002568,2.0
4,2022-05-01 16:49:00.400,0.002566,2.0
...,...,...,...
2896,2022-05-01 16:53:49.600,9.843215,3.0
2897,2022-05-01 16:53:49.700,9.844082,3.0
2898,2022-05-01 16:53:49.800,9.844949,3.0
2899,2022-05-01 16:53:49.900,9.845817,3.0


In [27]:
simulink_q = [0.2809089,-0.306867,-0.304860,-0.856727]
basilisk_q = [0.2405719,0.351735,0.219754,0.877562]

ang_error = error_angle_generator(basilisk_q,simulink_q)

ang_error


np.float64(12.1961531856319)

In [26]:
fig = px.scatter(error, x='Time', y='angle error', hover_data=['operation'])
fig.show()

# Plot Reference Profile Q from Basilisk against Simulink

## Load Simulink Data

In [156]:
data3 = f'../../ff1-aoc/simq.csv'

df_simulink = pd.read_csv(data3, header=0,names=['# Time (s)', 'q0', 'q1', 'q2', 'q3'])
df_simulink

,# Time (s),q0,q1,q2,q3
0,0.1,260300.0,330410.0,249190.0,872340.0
1,0.2,260260.0,330440.0,249190.0,872340.0
2,0.3,260230.0,330480.0,249190.0,872330.0
3,0.4,260200.0,330510.0,249190.0,872330.0
4,0.5,260160.0,330550.0,249190.0,872330.0
...,...,...,...,...,...
5294,529.5,7003.0,-536690.0,-146170.0,-830990.0
5295,529.6,7066.1,-536730.0,-146120.0,-830970.0
5296,529.7,7129.2,-536770.0,-146080.0,-830950.0
5297,529.8,7192.3,-536810.0,-146040.0,-830930.0


# Compare J2 Propagator PV against Orekit True PV

In [157]:
df2

,# Time (s),v_satPosJ2_calc[0],v_satPosJ2_calc[1],v_satPosJ2_calc[2]
0,0.0,0.000000e+00,0.000000e+00,0.000000e+00
1,0.1,0.000000e+00,0.000000e+00,0.000000e+00
2,0.2,0.000000e+00,0.000000e+00,0.000000e+00
3,0.3,0.000000e+00,0.000000e+00,0.000000e+00
4,0.4,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...
2396,239.6,-6.505212e+06,-1.748263e+06,1.774789e+06
2397,239.7,-6.505054e+06,-1.748111e+06,1.775513e+06
2398,239.8,-6.504896e+06,-1.747959e+06,1.776237e+06
2399,239.9,-6.504738e+06,-1.747808e+06,1.776961e+06


In [158]:
epoch2 = []
time = start_epoch
for i in range (len(df2)):
    time = start_epoch + df2['# Time (s)'][i]
    epoch2.append(pd.Timestamp(time.get_datetime()))
df2['# Time (s)'] = epoch2
df2 = df2[(df2['# Time (s)'] >= start_specific_timestamp) & (df2['# Time (s)'] <= end_specific_timestamp)]
df2 = df2.reset_index(drop=True)
# df2 = df2.drop(columns=['Unnamed: 0'])

In [159]:
df1 = df_orekit.drop(columns=['operation','quaternion','quaternion_conjugate'])
df1

KeyError: "['quaternion', 'quaternion_conjugate'] not found in axis"

: 

In [ ]:
errorp = pd.DataFrame()
errorp['time'] = df2['# Time (s)']
errorp['px'] = df2['v_satPosJ2_calc[0]'] - df1['px']
errorp['py'] = df2['v_satPosJ2_calc[1]'] - df1['py']
errorp['pz'] = df2['v_satPosJ2_calc[2]'] - df1['pz']
# errorp['vx'] = df2['vx'] - df1['vx']
# errorp['vy'] = df2['vy'] - df1['vy']
# errorp['vz'] = df2['vz'] - df1['vz']
errorp

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=errorp['time'], y=errorp['px']))
fig.add_trace(go.Scatter(x=errorp['time'], y=errorp['py']))
fig.add_trace(go.Scatter(x=errorp['time'], y=errorp['pz']))
plot_font = dict(family="Courier New, monospace", size=18, color="RebeccaPurple")
fig.update_layout(
    font_family="Courier New",
    font_color="black",
    title_font_family="Times New Roman",
    title_font_color="red",
    legend_title_font_color="green",
    title = 'Error Plot: J2 Prop vs Orekit True'
)

# J2 Propagator Error Analysis

## J2 Propagator - Simulink vs Basilisk

In [ ]:
%%script false

# Load simulink data into dataframe
matfile1 = f'../data/J2possimulink.mat'
matfile2 = f'../data/J2velsimulink.mat'

with h5py.File(matfile1, 'r') as file1:
    data1 = file1['j2pos'][:]
    print(list(file1.keys()))

with h5py.File(matfile2, 'r') as file2:
    data2 = file2['j2vel'][:]
    print(list(file2.keys()))

df8 = pd.DataFrame(data1)
df9 = pd.DataFrame(data2)
df8 = pd.DataFrame(df8, columns=['# Time (s)','v_J2X','v_J2Y','v_J2Z'])
df1_combined = pd.merge(df8,df9,on='# Time (s)', how='right')
epoch4 = []
for i in range (len(df1_combined)):
    time = start_epoch + df1_combined['# Time (s)'][i]
    epoch4.append(pd.Timestamp(time.get_datetime()))
df1_combined['# Time (s)'] = epoch4
df1_combined = df1_combined[(df1_combined['# Time (s)'] >= start_specific_timestamp) & (df1_combined['# Time (s)'] <= end_specific_timestamp)]
df1_combined = df1_combined.dropna()
df1_combined = df1_combined.reset_index(drop=True)

df8

In [ ]:
%%script false
df2

In [25]:
%%script false
df6['# Time (s)'] = df6['# Time (s)'].astype('float64')
epoch5 = []
for i in range (len(df6)):
    time = start_epoch + df6['# Time (s)'][i]
    epoch5.append(pd.Timestamp(time.get_datetime()))
df6['# Time (s)'] = epoch5
df6 = df6[(df6['# Time (s)'] >= start_specific_timestamp) & (df6['# Time (s)'] <= end_specific_timestamp)]
df2_combined = pd.merge(df2,df6,on='# Time (s)', how='right')
df2_combined = df2_combined.dropna()
df2_combined = df2_combined.reset_index(drop=True)

specific_time = '2022-05-01 16:50:00'

# # Filter the row where the 'time' column matches the specific time
# filtered_row = df2_combined[df2_combined['# Time (s)'] == specific_time]
# filtered_row

In [ ]:
%%script false
errorv = pd.DataFrame()
errorv['time'] = df2_combined['# Time (s)']
errorv['px'] = df2_combined['v_satPosJ2_calc[0]'] - df1_combined['v_J2X']
errorv['py'] = df2_combined['v_satPosJ2_calc[1]'] - df1_combined['v_J2Y']
errorv['pz'] = df2_combined['v_satPosJ2_calc[2]'] - df1_combined['v_J2Z']
errorv['vx'] = df2_combined['v_satVelJ2_calc[0]'] - df1_combined['v_J2vx']
errorv['vy'] = df2_combined['v_satVelJ2_calc[0]'] - df1_combined['v_J2vy']
errorv['vz'] = df2_combined['v_satVelJ2_calc[0]'] - df1_combined['v_J2vz']
errorv

In [ ]:
%%script false

fig = go.Figure()
fig.add_trace(go.Scatter(x=errorv['time'], y=errorv['px']))
fig.add_trace(go.Scatter(x=errorv['time'], y=errorv['py']))
fig.add_trace(go.Scatter(x=errorv['time'], y=errorv['pz']))

# fig = px.line(errorv, x="time", y="px")
# fig = px.line(errorv, x="time", y="py")
# fig2 = px.scatter(errorv, x="time", y="pz")
# fig.show()
# # fig1.show()
# fig2.show()

## J2 Propagator - Simulink vs Orekit

In [ ]:
%%script false
errort = pd.DataFrame()
errort['time'] = df1_combined['# Time (s)']
errort['px'] = df1_combined['v_J2X'] - df1['px']
errort['py'] = df1_combined['v_J2Y'] - df1['py']
errort['pz'] = df1_combined['v_J2Z'] - df1['pz']
# errort['vx'] = df1_combined['v_J2vx'] - df1_combined['v_J2vx']
# errort['vy'] = df1_combined['v_J2vy'] - df1_combined['v_J2vy']
# errort['vz'] = df1_combined['v_J2vz'] - df1_combined['v_J2vz']
errort

In [ ]:
%%script false
fig = go.Figure()
fig.add_trace(go.Scatter(x=errort['time'], y=errort['px']))
fig.add_trace(go.Scatter(x=errort['time'], y=errort['py']))
fig.add_trace(go.Scatter(x=errort['time'], y=errort['pz']))
